<a href="https://colab.research.google.com/github/sidms24/internship/blob/main/internship/june%20/week3_/Tuesday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.2 MB/s eta 0:00:00


In [9]:
import pandas as pd
from ast import pattern
from Bio import SeqIO
import re
from itertools import islice

In [11]:
!wget -O "/content/phenome_20250617.tsv" https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/phenome_20250617.tsv?token=GHSAT0AAAAAADFXXNIJR6SDV7RYNPH3JWTS2CSPFUQ

--2025-06-18 08:30:58--  https://raw.githubusercontent.com/sidms24/internship/refs/heads/main/phenome_20250617.tsv?token=GHSAT0AAAAAADFXXNIJR6SDV7RYNPH3JWTS2CSPFUQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169987 (166K) [text/plain]
Saving to: ‘/content/phenome_20250617.tsv’

/content/phenome_20 100%[===================>] 166.00K  --.-KB/s    in 0.01s   

2025-06-18 08:30:58 (11.0 MB/s) - ‘/content/phenome_20250617.tsv’ saved [169987/169987]



In [12]:
df = pd.read_csv('/content/phenome_20250617.tsv', sep='\t')

In [ ]:
profile = ProfileReport(df, title="Profiling Report")

In [13]:
df = df[['#NGB ID',
 'Accession ID',
 'CNSA ID',
 'Seed color']]

In [14]:
df = df[df['Seed color']!='--'].reset_index()

In [15]:
df

,index,#NGB ID,Accession ID,CNSA ID,Seed color
0,0,NGBLS00001,TKI-001,CNS0047843,white
1,1,NGBLS00002,TKI-002,CNS0047569,black
2,2,NGBLS00003,TKI-003,CNS0047893,white
3,3,NGBLS00004,TKI-004,CNS0047868,white
4,4,NGBLS00005,TKI-006,CNS0047930,white
...,...,...,...,...,...
378,437,NGBLS00451,TKI-469,CNS0047527,black
379,438,NGBLS00454,TKI-473,CNS0047599,black
380,439,NGBLS00455,TKI-474,CNS0047686,black
381,440,NGBLS00456,TKI-476,CNS0252039,black


In [25]:
!wget -O '/content/CL100066450_L01_525_1.fq.gz' https://huggingface.co/datasets/DNA-LLM/lettuce_db/resolve/main/CL100066450_L01_525_1.fq.gz

--2025-06-18 08:36:15--  https://huggingface.co/datasets/DNA-LLM/lettuce_db/resolve/main/CL100066450_L01_525_1.fq.gz
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.103, 18.239.50.80, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f3/14/f314c9a5f7ece36e1fe4d286175379875001a233e5d36183106795fbcadc1071/e713af006f08f6ff98cb8ca3ab2ad0df2126a73a76740922731d41cbf7e97ab9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27CL100066450_L01_525_1.fq.gz%3B+filename%3D%22CL100066450_L01_525_1.fq.gz%22%3B&response-content-type=application%2Fgzip&Expires=1750239375&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDIzOTM3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2YzLzE0L2YzMTRjOWE1ZjdlY2UzNmUxZmU0ZDI4NjE3NTM3OTg3NTAwMWEyMzNlNWQzNjE4MzEwNjc5NWZiY2FkYzEwNzEvZTcxM2FmMDA2ZjA4ZjZmZjk4

In [27]:
!gunzip /content/CL100066450_L01_525_1.fq.gz

In [139]:
from ast import pattern
from Bio import SeqIO
from itertools import islice, product
import re
import gzip
from collections import Counter


def kmers(sequence, ksize):
    k_mer_counts = Counter()
    for i in range(len(sequence) - ksize + 1):
        k_mer = sequence[i:i+ksize]
        k_mer_counts[k_mer] += 1
    return k_mer_counts
bases = ['A', 'C', 'G', 'T']
def generate_all_kmers(k):
    return sorted(["".join(p) for p in product(bases, repeat=k)])

all_kmers_vocab = {i:generate_all_kmers(i) for i in range(10)}
def parser(filepath):
  pattern = re.compile(r'(CL\d+)(L\d+)(C\d+R\d+)_(\d+)/(\d)')
  with open(filepath) as handle:
      for record in SeqIO.parse(handle, "fastq"):
        match = pattern.match(record.id)
        if match:
          yield {'sample id':match.group(1), "lane":match.group(2),
                 'col_row':match.group(3), 'index':match.group(4),
                 'read':match.group(5), 'sequence':str(record.seq)}



def read_weep(filepath,  sequencefolder, chunk_size = 100000, ksize = 2):
  recs = parser(filepath)
  chunkcount= 0
  current_all_kmers = all_kmers_vocab[ksize]

  while True:
    chunk = list(islice(recs, chunk_size))

    if not chunk:
      break
    a = []
    print(f"Processing a chunk of {len(chunk)} records...")
    for record in chunk:
      sequence = record['sequence']
      kmers_counts = kmers(sequence, ksize)


      record_features = {
            'sample_id': record['sample id'],
            'lane': record['lane'],
            'col_row': record['col_row'],
            'index': record['index'],
            'read': record['read'],
            'original_sequence_length': len(sequence)
        }


      for kmer in current_all_kmers:
        record_features[f'{kmer}_count'] = kmers_counts[kmer] / 100 if kmer in kmers_counts else 0

      a.append(record_features)
    chunkcount+=1
    df_chunk = pd.DataFrame(a)
    outputname =  sequencefolder + '/' + f'chunk_{chunkcount}.parquet'
    df_chunk.to_parquet(outputname)
    print(f"Processed chunk {chunkcount} and saved as {outputname}")

In [122]:
len(all_kmers_vocab[6])

4096

In [77]:
filepath ='/content/CL100066450_L01_525_1.fq'

In [147]:
read_weep(filepath,'/content/Sequence data/Sequence 1', chunk_size = 1000000, ksize = 2)

Processing a chunk of 1000000 records...
Processed chunk 1 and saved as /content/Sequence data/Sequence 1/chunk_1.parquet
Processing a chunk of 1000000 records...
Processed chunk 2 and saved as /content/Sequence data/Sequence 1/chunk_2.parquet
Processing a chunk of 1000000 records...
Processed chunk 3 and saved as /content/Sequence data/Sequence 1/chunk_3.parquet
Processing a chunk of 1000000 records...
Processed chunk 4 and saved as /content/Sequence data/Sequence 1/chunk_4.parquet
Processing a chunk of 1000000 records...
Processed chunk 5 and saved as /content/Sequence data/Sequence 1/chunk_5.parquet
Processing a chunk of 1000000 records...
Processed chunk 6 and saved as /content/Sequence data/Sequence 1/chunk_6.parquet
Processing a chunk of 1000000 records...
Processed chunk 7 and saved as /content/Sequence data/Sequence 1/chunk_7.parquet
Processing a chunk of 1000000 records...
Processed chunk 8 and saved as /content/Sequence data/Sequence 1/chunk_8.parquet
Processing a chunk of 10

In [152]:
chunk33 = pd.read_parquet('/content/Sequence data/Sequence 1/chunk_2.parquet')

In [151]:
chunk2

,sample_id,lane,col_row,index,read,original_sequence_length,AA_count,AC_count,AG_count,AT_count,...,CG_count,CT_count,GA_count,GC_count,GG_count,GT_count,TA_count,TC_count,TG_count,TT_count
0,CL100066450,L1,C001R052,180289,1,100,0.12,0.05,0.06,0.06,...,0.01,0.11,0.06,0.03,0.06,0.01,0.01,0.14,0.03,0.06
1,CL100066450,L1,C001R052,180308,1,100,0.10,0.06,0.09,0.06,...,0.08,0.03,0.12,0.06,0.08,0.04,0.02,0.06,0.04,0.03
2,CL100066450,L1,C001R052,180314,1,100,0.16,0.06,0.05,0.08,...,0.01,0.07,0.05,0.04,0.05,0.04,0.10,0.02,0.07,0.13
3,CL100066450,L1,C001R052,180322,1,100,0.10,0.05,0.06,0.07,...,0.06,0.04,0.09,0.07,0.09,0.04,0.03,0.04,0.08,0.11
4,CL100066450,L1,C001R052,180374,1,100,0.10,0.04,0.06,0.08,...,0.05,0.04,0.06,0.04,0.05,0.07,0.05,0.07,0.07,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,CL100066450,L1,C002R008,496796,1,100,0.05,0.02,0.09,0.10,...,0.03,0.07,0.10,0.06,0.07,0.04,0.06,0.07,0.08,0.08
999996,CL100066450,L1,C002R008,496811,1,100,0.02,0.13,0.03,0.06,...,0.03,0.09,0.02,0.06,0.05,0.02,0.09,0.04,0.04,0.03
999997,CL100066450,L1,C002R008,496827,1,100,0.05,0.04,0.07,0.16,...,0.00,0.00,0.10,0.03,0.04,0.06,0.08,0.03,0.11,0.12
999998,CL100066450,L1,C002R008,496850,1,100,0.01,0.06,0.01,0.12,...,0.04,0.03,0.06,0.01,0.01,0.10,0.07,0.05,0.12,0.22
